# Module_Att

## 1.4 - Module Attendance

## Task Description

Write a Python program to display the attendance of a module in a particular
week. Given module code and week number, the program should produce a list
which shows attendance of each session in the week. The attendance of each
session should be colour coded to highlight poor and good attendance. 

Firstly, all necessary modules are imported into the notebook.

In [1]:
# import modules
import numpy as np
import pandas as pd
import sqlite3

## Define module code  and week value variables

In [2]:
def define_variables_2():
    """
    This function prompts the user to enter a module code and week value.
    
    Args:
    None
    
    Returns:
    module_code, week_value: Users input for each variable.
    """
    module_code = input("The module code is (e.g. 22COA122ModuleRegister,\n\
    22COA111ModuleRegister):")
    week_value = input ("The week value is (e.g. W1, W2, W3, W5):")
    return(module_code, week_value)

## Read and modify dataframe

Note: While the initial dataframe has Na values for students who have authorised absences, I have opted to use fillna(0) to view these as absent regardless of the circumstances. While I understand that this is not the most optimal method, this is the logic I have used in order to determine how the students attendances are calculated.

This same logic will be used consistently throughout the entire notebook.

In [3]:
def read_sql_to_pandas(module_code):
    """
    This function reads the attendance and session data from the SQLite
    database, returning them as pandas dataframes.
    There is also a validity check to ensure the module code is entered
    correctly.
    
    Args:
    module_code (str): The code of the module to read from the SQLite
    database.
    
    Returns:
    df_attendance, df_sessions (pd.Dataframe): Dataframes containing attendance
    and session data for the specified module code.
    """
    conn = sqlite3.connect("CWDatabase.db")
    try:
        # Read in attendance data for relevant module
        df_attendance = pd.read_sql(f"SELECT * FROM '{module_code}_attendance'",
                                    conn, index_col = ['s_id'])
        # Read in sessions data for relevant module
        df_sessions = pd.read_sql(f"SELECT * FROM '{module_code}_sessions'",
                                  conn, index_col = ['s_id'])
    except sqlite3.OperationalError:
        print(f"Error: No table for module code '{module_code}' \n\
        found in the database.")
   
    conn.close()
    return(df_attendance, df_sessions)

def df_fillna_0(df_attendance):
    """
    This function fills NaN values with 0 in the attendance dataframe.
    
    Args:
    df_attendance (pd.DataFrame): Attendance data DataFrame.
    
    Returns:
    df_attendance (pd.DataFrame): Attendance dataframe with 0 instead of Na.
    """
    df_attendance = df_attendance.fillna(0)
    return(df_attendance)

## Calculate average attendance per session

In [4]:
def avg_session_att(df_attendance):
    """
    This function creates a new row in the dataframe which shows the mean
    of every session.
    It then transposes the dataframe.
    
    Args:
    df_attendance (pd.DataFrame): Attendance data DataFrame.
    
    Returns:
    df_attendance (pd.DataFrame): Attendance dataframe with the extra
    row showing avg_att % for every session.
    """
    # Calculate mean attendance for each session
    df_attendance.loc['avg_att'] = df_attendance.mean().mul(100)
    
    # Transpose the dataframe
    df_attendance = df_attendance.T
    return(df_attendance)
    

## Merge the modified attendance table with the sessions table

In [5]:
def merged_att_sesh_df(df_attendance, df_sessions):
    """
    This function merges the attendance and sessions table on the 's_id' column
    which is the index, to return a merged dataframe.
    
    Args:
    df_attendance (pd.DataFrame): Modified attendance dataframe
    df_sessions (pd.DataFrame): Dataframe containing session details throughout
    the semester.
    
    Returns:
    merged_df (pd.DataFrame): Merged dataframe containing attendance and session
    data together.
    """
    # Ensure both dataframes have their index in integer format
    df_attendance.index = df_attendance.index.astype(int)
    df_sessions.index = df_sessions.index.astype(int)
    
    # Merge both dataframes
    merged_df = pd.merge(df_attendance, df_sessions, left_index = True,
                         right_index = True, how = 'inner')
    return(merged_df)

## Create a new pandas df with only relevant columns from merged_df

In [6]:
def weekly_module_att(merged_df):
    """
    This function extracts relevant columns from the merged dataframe
    to create a new dataframe.
    
    Args:
    merged_df (pd.DataFrame): Merged dataframe containing attendance and 
    session details.
   
    Returns:
    module_weekly_att_df (pd.DataFrame): A new dataframe containing only
    relevant columns from the merged dataframe.
    """
    module_weekly_att_df = merged_df[['Week','Time', 'Room',
                                      'Lesson','avg_att']]
    
    return(module_weekly_att_df)

## Filter dataframe for the specified week

In [20]:
def module_att(module_weekly_att_df, week_value):
    """
    This function filters the dataframe showing average attendance for every
    session, in order to only show the rows of data for the specific week
    that the user requests to see.
    There is also a validity check to ensure the week value inputted
    by the user is a week number in the dataframe.
    
    Args:
    module_weekly_att_df (pandas.DataFrame): Dataframe containing session
    details and average attendance for every session.
   
    Returns:
    module_att_wk (pd.DataFrame): A filtered version of the previous dataframe
    only showing data for the relevant week selected.
    """
    if not module_weekly_att_df['Week'].isin([week_value]).any():
        print("Error: The entered week value is not a valid option in the dataframe.")
        return None
    # Select rows where the 'Week' column has the value entered by the user
    module_att_wk = module_weekly_att_df[module_weekly_att_df['Week'] ==
                                         week_value]
    return(module_att_wk)

## Colour code module_att_wk

The criteria set for determining attendance colour is:

If attendance is over 70: Good (Green)

If attendance is between 40-70: Moderate (Yellow)

If attendance is below 40: Bad (Red)

In [8]:
def highlight_attendance_1(value):
    """
    This function takes a numerical value and assigns a colour 
    (green, orange, or red) based on the attendance value.
    
    Args:
    value (float): Attendance values
    
    Returns:
    str: A string showing the colour coding, ready to be used in
    a styling command.
    """
    if value < 40:
        color = 'red'
    elif value < 70:
        color = 'orange'
    else:
        color = 'green'
    return 'color: {}'.format(color)

# Main Code

In [9]:
def main_module_att():
    """
    This function is the main function that calls all the other functions
    previously created in this notebook.
    It then uses a styler to apply the colour coding function to the avg_att
    column of the dataframe containing session details and average attendance.
    
    Args:
    None
    
    Returns:
    None
    """
    module_code, week_value = define_variables_2()
    df_attendance, df_sessions = read_sql_to_pandas(module_code)
    df_attendance = df_fillna_0(df_attendance)
    df_attendance = avg_session_att(df_attendance)
    merged_df = merged_att_sesh_df(df_attendance, df_sessions)
    module_weekly_att_df = weekly_module_att(merged_df)
    module_att_wk = module_att(module_weekly_att_df, week_value)
    
    # using highlight_attendance function
    print(f"\nAttendance record for {module_code} Week {week_value}:")
    att_table_colour = module_att_wk.style.applymap(highlight_attendance_1,
                                                    subset=['avg_att'])
    
    display(att_table_colour)

## Main Test Code 1 - 22COA122

The output of this main test code will show every session in the week and module you specified, where avg_att will be highlighted either green, yellow, or red depending on if it is good, moderate, or bad attendance according to the criteria I have set.

In [10]:
main_module_att()
# using 22COA122ModuleRegister

The module code is (e.g. 22COA122ModuleRegister,
    22COA111ModuleRegister):22COA122ModuleRegister
The week value is (e.g. W1, W2, W3, W5):W1

Attendance record for 22COA122ModuleRegister Week W1:


,Week,Time,Room,Lesson,avg_att
1,W1,14:00 - 16:00,CC012,Lecture,85.427136
2,W1,09:00 - 11:00,N001...,Computer Lab,42.211055
3,W1,11:00 - 13:00,N001...,Computer Lab,30.150754


## Main Test Code 2 - 22COA111

In [11]:
main_module_att()
# using 22COA111ModuleRegister

The module code is (e.g. 22COA122ModuleRegister,
    22COA111ModuleRegister):22COA111ModuleRegister
The week value is (e.g. W1, W2, W3, W5):W1

Attendance record for 22COA111ModuleRegister Week W1:


,Week,Time,Room,Lesson,avg_att
1,W1,09:00 - 10:00,U020,Lecture,71.859296
2,W1,10:00 - 11:00,SMB014,Lecture,63.316583


# Testing all functions in the Notebook

Running the function (define_variables_2) prompts the user to input values for module code and week value. These user defined variables are then referred to throughout the rest of the code. Printing these variables show that python has successfully assigned the user inputted value to the variable name.

In [12]:
module_code, week_value = define_variables_2()
print(module_code, week_value)

The module code is (e.g. 22COA122ModuleRegister,
    22COA111ModuleRegister):22COA111ModuleRegister
The week value is (e.g. W1, W2, W3, W5):W1
22COA111ModuleRegister W1


Running the function (read_sql_to_pandas) creates two pandas dataframes for the module (attendance and sessions respectively). These dataframes are extracted from the SQLite database 'CWDatabase.db'.

In [13]:
df_attendance, df_sessions = read_sql_to_pandas(module_code)
display(df_attendance, df_sessions)

,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,None,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,NaN,NaN,1.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0
239,1.0,0.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.0
240,1.0,1.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0


,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
7,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,,S1
8,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1


Running the function (df_fillna_0) simply fills all Na values in the attendance table with 0. The logic behind this is that regardless of whether an absence is authorised or not, it is assumed that that student is still marked as absent, so is appointed a value of 0.

In [14]:
df_attendance = df_fillna_0(df_attendance)
display(df_attendance)

,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,1.0,1.0,0.0,0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
239,1.0,0.0,1.0,0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
240,1.0,1.0,1.0,0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


Running the function (avg_session_att) creates a new row in the attendance table which calculates the average attendance over the range of all students for each session. It then transposes the table so the avg_att is now a column at the far right of the table as shown below.

In [15]:
df_attendance = avg_session_att(df_attendance)
display(df_attendance)

s_id,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,avg_att
1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,71.859296
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,63.316583
3,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,69.346734
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,80.904523
6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,60.301508
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,78.391960
9,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,60.804020
11,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,77.386935
12,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,63.316583


Running the function (merged_att_sesh_df) merges the attendance and sessions dataframes together into one large dataframe shown below. This merged dataframe also contains the avg_att column.

In [16]:
merged_df = merged_att_sesh_df(df_attendance, df_sessions)
display(merged_df)

,0,1,2,3,4,5,6,7,8,9,...,241,242,avg_att,Week,Day,Time,Lesson,Room,Excluded,Semester
1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,71.859296,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,63.316583,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,69.346734,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,80.904523,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,60.301508,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,78.391960,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,60.804020,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
11,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,77.386935,W4,Thursday (27-10-2022),14:00 - 15:00,Lecture,CC011,,S1
12,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,63.316583,W5,Wednesday (02-11-2022),10:00 - 11:00,Lecture,SMB014,,S1


Running the function (weekly_module_att) creates a new pandas dataframe from the merged dataframe. This new dataframe only contains relevant columns such as week, time, room, lesson and most importantly average attendance for every session.

In [17]:
module_weekly_att_df = weekly_module_att(merged_df)
display(module_weekly_att_df)

,Week,Time,Room,Lesson,avg_att
1,W1,09:00 - 10:00,U020,Lecture,71.859296
2,W1,10:00 - 11:00,SMB014,Lecture,63.316583
3,W2,10:00 - 11:00,SMB014,Lecture,69.346734
4,W2,12:00 - 13:00,,Tutorial,0.000000
5,W2,14:00 - 15:00,CC011,Lecture,80.904523
6,W3,10:00 - 11:00,SMB014,Lecture,60.301508
8,W3,14:00 - 15:00,CC011,Lecture,78.391960
9,W4,10:00 - 11:00,SMB014,Lecture,60.804020
11,W4,14:00 - 15:00,CC011,Lecture,77.386935
12,W5,10:00 - 11:00,SMB014,Lecture,63.316583


Running the function (module_att) filters the dataframe above to only show data values for a particular week which has been specified by the user.

In [18]:
module_att_wk = module_att(module_weekly_att_df, week_value)
display(module_att_wk)

,Week,Time,Room,Lesson,avg_att
1,W1,09:00 - 10:00,U020,Lecture,71.859296
2,W1,10:00 - 11:00,SMB014,Lecture,63.316583


Running the code below which has been extracted from the main code function, the previous table has a styler applied to it, using the highlight_attendance function, so that the values of attendance are coloured according to the criteria set for whether attendance is good, moderate, or bad.

In [19]:
print(f"\nAttendance record for {module_code} Week {week_value}:")
att_table_colour = module_att_wk.style.applymap(highlight_attendance_1,
                                                subset=['avg_att'])

display(att_table_colour)


Attendance record for 22COA111ModuleRegister Week W1:


,Week,Time,Room,Lesson,avg_att
1,W1,09:00 - 10:00,U020,Lecture,71.859296
2,W1,10:00 - 11:00,SMB014,Lecture,63.316583
